In [14]:
#라이브러리 작성
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
import numpy as np
# import pandas as pd
from keras.models import *
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D, concatenate, Activation,Conv2DTranspose
from keras.layers import BatchNormalization
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, History
from keras import backend as keras
from keras.utils.multi_gpu_utils import multi_gpu_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model
import numpy as np
import os
import glob
import shutil
from keras import backend as K
# import matplotlib.pyplot as plt
from metrics import * # file
from losses import * #file

# import pandas as pd
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import precision_recall_fscore_support
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import r2_score
# import cv2
from keras.models import load_model
import os
from keras.callbacks import EarlyStopping
from tensorflow.compat.v1.keras import backend as K


## img ##
img_rows, img_cols = 1024,1024
img_type = 'png'

In [10]:
#U-NET
def get_unet(img_rows, img_cols):
    inputs = Input((img_rows, img_cols,1))
    conv1 = Conv2D(32, (3, 3), activation=None, padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(32, (3, 3), activation=None, padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation=None, padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(64, (3, 3), activation=None, padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation=None, padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(128, (3, 3), activation=None, padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation=None, padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(256, (3, 3), activation=None, padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation=None, padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(512, (3, 3), activation=None, padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation=None, padding='same')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)
    conv6 = Conv2D(256, (3, 3), activation=None, padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation=None, padding='same')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)
    conv7 = Conv2D(128, (3, 3), activation=None, padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation=None, padding='same')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Activation('relu')(conv8)
    conv8 = Conv2D(64, (3, 3), activation=None, padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Activation('relu')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation=None, padding='same')(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)
    conv9 = Conv2D(32, (3, 3), activation=None, padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    return model

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)


In [11]:
#데이터 로더 및 normalization
    def load_train_data(train_npy_path, mask_npy_path):

        print('-'*30)
        print('%s data load...'%i)


        print('-'*30)
        print('load train images...')
        print('-'*30)

        imgs_train = np.load(train_npy_path)
        imgs_mask_train = np.load(mask_npy_path)

        imgs_train = imgs_train.astype('float32')
        imgs_mask_train = imgs_mask_train.astype('float32')
        print('img : ', imgs_train.max())
        print('mask : ',imgs_mask_train.max())

        print('-'*30)
        print('normalization start...')
        print('-'*30)
        imgs_train = imgs_train/255.0

        imgs_mask_train[imgs_mask_train <= 127] = 0
        imgs_mask_train[imgs_mask_train > 127] = 1

        print('img : ',imgs_train.max())
        print('mask : ',imgs_mask_train.max())

        return imgs_train, imgs_mask_train

    def load_test_data(test_npy_path):
        print('-'*30)
        print('load test images...')
        print('-'*30)

        imgs_test = np.load(test_npy_path)
        imgs_test = imgs_test.astype('float32')
        imgs_test /= 255

        return imgs_test

    def load_data(train_npy_path, mask_npy_path, test_npy_path):
        imgs_train, imgs_mask_train = load_train_data(train_npy_path, mask_npy_path)
        imgs_test = load_test_data(test_npy_path)
        print(imgs_train.shape)
        print(imgs_mask_train.shape)
        print(imgs_test.shape)
        return (imgs_train, imgs_mask_train, imgs_test)

# 학습 시작

In [12]:
pwd

'/home/ubuntu/gcubme4/Workspace/HM_LEE/spine_scoliosis/total_code'

In [ ]:
tensorboard --logdir=./9th_data_aug/result/0/log --port=8895 --host=10.2.52.54

In [ ]:
for i in range(1):
    ## npy file #`#
    npy_path = '../7th_total/data/%s/aug/npy/'%i

    #학습저장 경로
    check_model_path = '../7th_total/result/%s/check/'%i
    model_path='../7th_total/result/%s/model/'%i
    predict_path = '../7th_total/result/%s/npy/'%i
    result_save_paths="../7th_total/result/%s/img/compare/"%i
    pred_img_path = '../7th_total/result/%s/img/mask/'%i
    logdir=total_path+"result/%s/log/"%i + datetime.now().strftime("%Y%m%d-%H%M%S")
    

    imgs_train, imgs_mask_train, imgs_test = load_data(npy_path+'train_1024.npy', npy_path+'label_1024.npy', npy_path+'test_1024.npy')

    
    # multi-GPU

    #학습 시작
    print('-'*30)
    print("load unet model")
    print('-'*30)
    
#     config = tf.compat.v1.ConfigProto()
#     config.gpu_options.per_process_gpu_memory_fraction=0.8
#     K.set_session(tf.compat.v1.Session(config=config))
    
    model = get_unet(img_rows, img_cols)

    model = multi_gpu_model(model,gpus=2)
    
    model.compile(optimizer=Adam(lr=1e-3), loss='binary_crossentropy', metrics=[dice_coef_loss])
    model_checkpoint = ModelCheckpoint(check_model_path+'check_11_8_%s.h5'%(i), monitor='loss',verbose=1, save_best_only=True, save_weights_only=False)
#     model_checkpoint = ModelCheckpoint(check_model_path+'ckpt_{epoch:02d}.h5', monitor='loss',verbose=1, save_best_only=True)

#     tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, update_freq='epoch')
    
#     es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
#     model.fit(imgs_train, imgs_mask_train, batch_size=6, epochs=150, verbose=1, validation_split=0.2, shuffle=True, callbacks=[es,model_checkpoint])

    print('Fitting model...')
    model.fit(imgs_train, imgs_mask_train, batch_size=2, epochs=5, verbose=1, validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])
    
    print('save model')
    model.save(model_path+'model_11_8.h5')

#     print('predict test data')
#     imgs_mask_test = model.predict(imgs_test, batch_size=1, verbose=1)
#     np.save(predict_path+'predict_11_8_%s.npy'%i, imgs_mask_test)
    
    
    %matplotlib inline

    fig, loss_ax = plt.subplots()

    acc_ax = loss_ax.twinx()

    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

    acc_ax.plot(hist.history['dice_coef_loss'], 'b', label='train dice_coef_loss')
    acc_ax.plot(hist.history['val_dice_coef_loss'], 'g', label='val dice_coef_loss')

    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    acc_ax.set_ylabel('dice_coef_loss')

    loss_ax.legend(loc='upper left')
    acc_ax.legend(loc='lower left')

    plt.show()

    